In [19]:
import numpy as np
from BPmodule.BPGen import CreateData
from BPnumba.GeneticOperators import CreateHeuristicPob,Ind
from BPnumba.BPPH import DBLF2,create_Bin, Bin, Placement, Overlap, AddBox
from numba import njit, objmode, prange
import time
from numba.typed import List as NumbaList
from typing import List
from BPmodule.DataLecture import ExpotarJSON,PossiblePositions
from BPnumba.PqueueNumba import CreatePriorityQueue,PQVector,listP_type


In [28]:
contenedor,data = CreateData(algorithm=2,problem=2)

PATHBOX = "C:/Users/nicoo/BPP/Assets/bpp.json"
PATHPOINT = "C:/Users/nicoo/BPP/Assets/points.json"

In [16]:
datos = data[10]
n = len(datos)
sameInstance = CreateHeuristicPob(50,datos,True)
n

25

In [37]:
ind = Ind(NumbaList(sameInstance[0]))
ind.genome

ListType[int64]([3, 2, 5, 4, 10, 1, 8, 16, 14, 9, 12, 21, 13, 23, 25, 6, 11, 20, 7, 15, 17, 22, 24, 19, 18, ...])

In [45]:
@njit
def DBLF(bin:Bin, itemsToPack:List[int], BoxesData:List[List[int]], maxIter:int=0):
    lstP = CreatePriorityQueue(NumbaList([0,2,1]))
    lstP.push(NumbaList([0,0,0]))
    for i in np.arange(len(itemsToPack)):
        for j in np.arange(lstP.size()):
            pt = lstP.getPt(j)
            boxID = itemsToPack[i]
            itemV = BoxesData[boxID-1]
            if  Placement(bin.dimensions,NumbaList([pt[0]+itemV[0],pt[1]+itemV[1],pt[2]+itemV[2]])):
                overlap = Overlap(pt,boxID,BoxesData, bin.getBoxes(),bin.getPositions())
                if not overlap:
                    lstP.delPt(j)
                    AddBox(lstP,bin,pt,boxID,itemV,BoxesData)           
                    break
        if i == maxIter:
            break
    print(lstP.ToList())
    with objmode():
        PossiblePositions(PATHPOINT,lstP.ToList())

In [53]:
bin = create_Bin(NumbaList(contenedor))
DBLF(bin,ind.genome,datos,4)
print(bin.getLoadVol()/np.prod(contenedor))
print(bin.getN(),bin.getBoxes())
ExpotarJSON(PATHBOX,bin.getPositions(),bin.getBoxes(),datos,contenedor)

[[0, 38, 42, ...], [0, 47, 45, ...], [0, 0, 47, ...], [35, 19, 45, ...], ...]
0.91648
5 [3, 2, 5, 4, 10, ...]
